Dev notebook for scraping a list of PC games off of Metacritic. This list of games will get stored in a postgres table and be used later for scraping review data of each game.

In [1]:
import urllib.request
from bs4 import BeautifulSoup
from datetime import datetime
from time import sleep

In [5]:
user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
url = "http://www.metacritic.com/browse/games/release-date/available/pc/date?view=condensed&page=" # iterate through page numbers
headers={'User-Agent':user_agent,} 

In [28]:
def parse_game_titles(text_data):
    """
    Retrieves all the info associated with each game from input text
    
    :param text_data: text from a single <li> tag with class 'product game_product'
    :return: a dict of the form {url:game_url_tail, title:official_title}
    """
    atag = text_data.find('a', href=True)
    url_tail = atag['href']
    title = atag.get_text().strip()
    
    return {'url':url_tail, 'title':title}
    

In [98]:
def paginate_games(url, headers, pagelim = float('inf')):
    """
    Loops through pages and returns a list of games on each page
    
    :param url: url with all text except the page number
    :param headers: headers to include in the request
    :param pagelim: how many pages to paginate through
    
    :return: a list of game names and their data
    """
    
    gamelist = []
    i = 0
    while i <= pagelim:
        i += 1
        request = urllib.request.Request(url + str(i), None, headers)
        pagedata = urllib.request.urlopen(request).read()
        soup = BeautifulSoup(pagedata, "lxml")
        
        games_in_page = soup.findAll('li', attrs={'class': 'product game_product'})
        if not games_in_page:
            print('Explored ' + str(i) + ' pages before stopping')
            break
        
        for game in games_in_page:
            gamelist.append(parse_game_titles(game))
        
        sleep(2)
    
    return gamelist

In [29]:
g = paginate_games(url, headers)

In [19]:
g[1]

('Chess3D', '/game/pc/chess3d')

## Build Database

In [95]:
import psycopg2

conn=psycopg2.connect("dbname='scrapegame' user='rickwolf' password='mypass'")
conn.autocommit = True
cur = conn.cursor()

In [96]:
insert_cmd = """INSERT INTO games 
    (gametitle, metacriticurl)
SELECT %(title)s, %(url)s
WHERE
    NOT EXISTS (
        SELECT metacriticurl FROM games WHERE metacriticurl=%(url)s
    );
"""

In [80]:
print(insert_cmd % test[0])

INSERT INTO games (gametitle, metacriticurl)
SELECT 'Blink', '/game/pc/blink'
WHERE
    NOT EXISTS (
        SELECT metacriticurl FROM games WHERE metacriticurl='/game/pc/blink'
    );



In [99]:
# populate games table

g = paginate_games(url, headers)
cur.executemany(insert_cmd, g)

Explored 114 pages before stopping
